### **Importing Libraries**

In [1]:
import keras
import tensorflow as tf
import cv2
import glob as gb
import os
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="whitegrid")
import cv2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
s = 100

### **Downloading dataset from kaggle**

In [3]:
! pip install kaggle

In [4]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!ls

 datasets			    sample_data
'edible wild plants metadata.xls'   saved
 edible-wild-plants.zip		    saved.h5
 final_model_weights.hdf5	    vanilla_model_weights.hdf5
 kaggle.json


In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d gverzea/edible-wild-plants

edible-wild-plants.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!unzip edible-wild-plants.zip

Streaming output truncated to the last 5000 lines.
  inflating: datasets/dataset/Daisy Fleabane/34696730346_5f0c131e59_n.jpg  
  inflating: datasets/dataset/Daisy Fleabane/34701078235_4a770d14a1_n.jpg  
  inflating: datasets/dataset/Daisy Fleabane/34701198765_54aa641d7a_n.jpg  
  inflating: datasets/dataset/Daisy Fleabane/34718882165_68cdc9def9_n.jpg  
  inflating: datasets/dataset/Daisy Fleabane/34720703615_bdf1335d8b_n.jpg  
  inflating: datasets/dataset/Daisy Fleabane/34727863665_b00ac77266_n.jpg  
  inflating: datasets/dataset/Daisy Fleabane/34729724865_787c98299d_n.jpg  
  inflating: datasets/dataset/Daisy Fleabane/34733243845_29f1c30634_n.jpg  
  inflating: datasets/dataset/Daisy Fleabane/3474942718_c418dae6f1.jpg  
  inflating: datasets/dataset/Daisy Fleabane/3475870145_685a19116d.jpg  
  inflating: datasets/dataset/Daisy Fleabane/3483303007_42e3f90da7.jpg  
  inflating: datasets/dataset/Daisy Fleabane/3491933306_43cfe2cfbe.jpg  
  inflating: datasets/dataset/Daisy Fleabane/3494

### **Changing working directory**

In [10]:
print(os.listdir('/content/datasets/dataset'))
train_path = '/content/datasets/dataset'

['Chickweed', 'Downy Yellow Violet', 'Broadleaf Plantain', 'Dandellion', 'Wild Grape Vine', 'Knapweed', 'Wood Sorrel', 'Teasel', 'Joe Pye Weed', 'Spring Beauty', 'Fireweed', 'Alfalfa', 'Meadowsweet', 'Sunflower', 'Curly Dock', 'Mullein', 'Partridgeberry', 'New England Aster', 'Wild Black Cherry', 'Milk Thistle', 'Sheep Sorrel', 'Cattail', 'Fern Leaf Yarrow', 'Red Clover', 'Purple Deadnettle', 'Prickly Pear Cactus', 'Herb Robert', 'Mayapple', 'Asparagus', 'Henbit', 'Crimson Clover', 'Elderberry', 'Common Yarrow', 'Kudzu', 'Daisy Fleabane', 'Tea Plant', 'Forget Me Not', 'Vervain Mallow', 'Lambs Quarters', 'Coltsfoot', 'Field Pennycress', 'Cleavers', 'Pickerelweed', 'Supplejack Vine', 'Queen Annes Lace', 'Wild Bee Balm', 'Creeping Charlie', 'Wild Leek', 'Shepherds Purse', 'Japanese Knotweed', 'Blue Vervain', 'Peppergrass', 'Toothwort', 'Garlic Mustard', 'Evening Primrose', 'Pineapple Weed', 'Bull Thistle', 'Harebell', 'Mallow', 'Coneflower', 'Common Sow Thistle', 'Chicory']


### **Obtaining Dataset and marking images**

**Obtaining and labelling training set**

In [11]:
code = {"Asparagus": 1,"Blue Vervain":2,"Cattail": 3,"Chicory": 4,"Fireweed": 5,"Green Castor Bean": 6}

In [12]:
X = []
y = []
for folder in os.listdir(train_path):
    if folder in ["Asparagus","Blue Vervain","Cattail","Chicory","Fireweed","Green Castor Bean"]:
        for i in os.listdir(train_path + "/" + folder):
#             print(i)
            img = list(cv2.imread(train_path + "/" + folder + "/" + i))
            X.append(cv2.resize(np.array(img), (s, s)))
            y.append(code[folder])
print(len(X))

300


**Obtaining and labelling testing dataset**

In [13]:
test_path = '/content/datasets/dataset-test'
X_test = []
y_test = []
for folder in os.listdir(train_path):
    if folder in ["Asparagus","Blue Vervain","Cattail","Chicory","Fireweed","Green Castor Bean"]:
        for i in os.listdir(test_path + "/" + folder):
#             print(i)
            img = list(cv2.imread(test_path + "/" + folder + "/" + i))
            X_test.append(cv2.resize(np.array(img), (s, s)))
            y_test.append(code[folder])
print(len(X_test))

25


### **Scaling and converting dataset into required format**

In [14]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [15]:
X = np.array(X)
X = X/255.0
y = np.array(y)

### **Configuring Image Data Generator**

In [16]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=False)

datagen.fit(X)

### **Configuring model**

In [17]:
model1 = Sequential()
model1.add(Conv2D(filters = 40, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (s,s,3)))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Flatten(input_shape = (s,s)))
model1.add(keras.layers.Dense(6, activation='softmax'))


### **Model training and validation**

In [18]:
model1.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history1 = model1.fit(datagen.flow(X,y, batch_size=128),
                              epochs = 50, validation_data = (X_test,y_test),
                              verbose = 1, steps_per_epoch=2)

Epoch 1/50
2/2 [==============================] - 3s 929ms/step - loss: 3.7288 - accuracy: 0.2616 - val_loss: 1393.2887 - val_accuracy: 0.2000
Epoch 2/50
2/2 [==============================] - 1s 425ms/step - loss: 2.8241 - accuracy: 0.3140 - val_loss: 384.3338 - val_accuracy: 0.3200
Epoch 3/50
2/2 [==============================] - 1s 415ms/step - loss: 1.7710 - accuracy: 0.3281 - val_loss: 274.5726 - val_accuracy: 0.3200
Epoch 4/50
2/2 [==============================] - 1s 400ms/step - loss: 1.7450 - accuracy: 0.3008 - val_loss: 202.7540 - val_accuracy: 0.3600
Epoch 5/50
2/2 [==============================] - 1s 176ms/step - loss: 1.5818 - accuracy: 0.4419 - val_loss: 184.0519 - val_accuracy: 0.2400
Epoch 6/50
2/2 [==============================] - 1s 414ms/step - loss: 1.4731 - accuracy: 0.3779 - val_loss: 134.5755 - val_accuracy: 0.3200
Epoch 7/50
2/2 [==============================] - 1s 158ms/step - loss: 1.4195 - accuracy: 0.4070 - val_loss: 114.5075 - val_accuracy: 0.4000
Epoch

###**Making individual predictions**

In [20]:
#image input from user in flask
#img = cv2.imread("path to wherver image is stored")
img = list(cv2.imread("/content/datasets/dataset-test/BlueVervain/Blue_vervain_test1.jpg"))
img = cv2.resize(np.array(img), (100, 100))
# for i in X:
#   print(i.shape)
images = []
images.append(img)
images = np.array(images)
pred = model1.predict(images)
pred = pred[0]
if (pred[0] ==  1):
  print("Not Edible")
elif (pred[1] ==  1):
  print("Edible")
elif (pred[2] ==  1):
  print("Edible")
elif (pred[3] ==  1):
  print("Edible")
elif (pred[4] ==  1):
  print("Not Edible")
elif (pred[5] ==  1):
  print("Not Edible")
else:
  print("Plant not identified")

Edible


### **Saving model**

In [21]:
model1.save("saved")

INFO:tensorflow:Assets written to: saved/assets


In [24]:
model = keras.models.load_model("saved")
pred = model.predict(images)
pred = pred[0]
if (pred[0] ==  1):
  print("Not Edible")
elif (pred[1] ==  1):
  print("Edible")
elif (pred[2] ==  1):
  print("Edible")
elif (pred[3] ==  1):
  print("Edible")
elif (pred[4] ==  1):
  print("Not Edible")
elif (pred[5] ==  1):
  print("Not Edible")
else:
  print("Plant not identified")

Edible
